4 a)

$\nabla_W \ell(W)$$=\sum_{i=1}^n(\nabla_W\log|W|+\sum_{i=1}^d\nabla_W\log g'(w_j^Tx^{(i)}))$$=\sum_{i=1}^n(\frac{1}{|W|}|W|(W^{-1})^T+\sum_{j=1}^d \nabla_W (\log (\frac{1}{\sqrt{2\pi}}e^{-(w_j^Tx^{(i)})^2/2}))$$=\sum_{i=1}^n ((W^{-1})^T+\sum_{j=1}^d\nabla_W (-(w_j^Tx^{(i)})^2/2))$$=\sum_{i=1}^n ((W^{-1})^T-\frac{1}{2}\sum_{j=1}^d\nabla_W(w_j^Tx^{(i)})^2)$$=\sum_{i=1}^n ((W^{-1})^T-\frac{1}{2}\sum_{j=1}^d 2(w_j^Tx^{(i)})\nabla_W(w_j^Tx^{(i)})$$=\sum_{i=1}^n((W^{-1})^T-(W x^{(i)})(x^{(i)})^T$$=n (W^{-1})^T-WX^T X$.

Setting $\nabla_W\ell(W)=0$ gives $n(W^{-1})^T-WX^TX=0$, $n(W^{-1})^T=WX^TX$, $nW^{-1}=X^TXW^T$, and $W^TW=n(X^TX)^{-1}$. Given an orthonormal matrix $A$, $W^TA^TAW=W^TW$. So, the above equation couldn't uniquely determine $W$.

4 b)

$\nabla_W\ell(W)$$=\nabla_W (\sum_{i=1}^n (\log |W| + \sum_{j=1}^d \log (\frac{1}{2}exp(-|w_j^Tx^{(i)}|)))$$=\nabla_W(\sum_{i=1}^n \log |W|+\sum_{j=1}^d -|w_j^Tx^{(i)}|)$$=n (W^{-1})^T+\sum_{i=1}^n\sum_{j=1}^d \nabla_W-|w_j^Tx^{(i)}|$$=n(W^{-1})^T+\sum_{i=1}^n\sum_{j=1}^d-sgn(w^T_j x^{(i)})(x^{(i)})_j$ with $(x^{(i)})_j$ denoting the matrix with a single nonzero $j$th row being $x^{(i)}$. To maximize $\ell(W)$, set update to be $\nabla_W\ell(W)$ as above.

4 c)


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import scipy.io.wavfile
import os
import numpy as np

def update_W(W, x, learning_rate):
    """
    Perform a gradient ascent update on W using data element x and the provided learning rate.
    This function should return the updated W.
    Use the laplace distribiution in this problem.
    Args:
        W: The W matrix for ICA
        x: A single data element
        learning_rate: The learning rate to use
    Returns:
        The updated W
    """
    # *** START CODE HERE ***
    """
    Likelihood of a single data element x equals \log |W| + \sum_{j=1}^d \log g'(w_j^Tx^{(i)})
    stochastic gradient ascent update equals (W^{-1})^T + \sum_{j=1}^d -sgn(w_j^T x^{(i)}) (x^{(i)})_j
    """
    update = np.zeros(W.shape)
    update += np.linalg.inv(W)
    signs = np.sign(W @ x)
    update += - signs[:, None] * x[None, :]
    updated_W = W + learning_rate * update
    # *** END CODE HERE ***
    return updated_W

def unmix(X, W):
    """
    Unmix an X matrix according to W using ICA.
    Args:
        X: The data matrix
        W: The W for ICA
    Returns:
        A numpy array S containing the split data
    """
    # *** START CODE HERE ***
    S = (W @ X.T).T
    # *** END CODE HERE ***
    return S

Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('/content/drive/MyDrive/cs229/problem-sets/PS4/data/mix.dat')
    return mix

def save_W(W):
    np.savetxt('/content/drive/MyDrive/cs229/problem-sets/PS4/output/W.txt',W)

def save_sound(audio, name):
    scipy.io.wavfile.write('/content/drive/MyDrive/cs229/problem-sets/PS4/output/{}.wav'.format(name), Fs, audio)

def unmixer(X):
    M, N = X.shape
    W = np.eye(N)
    anneal = [0.1 , 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01 , 0.01, 0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    for lr in anneal:
        print(lr)
        rand = np.random.permutation(range(M))
        for i in rand:
            x = X[i]
            W = update_W(W, x, lr)
    return W

def main():
    # Seed the randomness of the simulation so this outputs the same thing each time
    np.random.seed(0)
    X = normalize(load_data())
    print(X.shape)
    for i in range(X.shape[1]):
        save_sound(X[:, i], 'mixed_{}'.format(i))
    W = unmixer(X)
    print(W)
    save_W(W)
    S = normalize(unmix(X, W))
    assert S.shape[1] == 5
    for i in range(S.shape[1]):
        if os.path.exists('split_{}'.format(i)):
            os.unlink('split_{}'.format(i))
        save_sound(S[:, i], 'split_{}'.format(i))

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(53442, 5)
Separating tracks ...
0.1
0.1
0.1
0.05
0.05
0.05
0.02
0.02
0.01
0.01
0.005
0.005
0.002
0.002
0.001
0.001
[[ 43.87436024  14.20333802  15.87371503  -9.14085174 -15.73432699]
 [ 15.05431262  26.69048344   3.35867168 -10.19254642  11.41007428]
 [ 14.83783755   2.31603796  20.13501041   8.39318277  -5.09756764]
 [ -9.95657858 -10.0089948    7.20614667  23.32738889   2.19641418]
 [-12.79418218  12.89666462  -3.00203513   2.47665334  33.20130964]]
